In [26]:
from __future__ import division, print_function

import numpy as np
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng
from pylab import plt

In [27]:
def J(Y, lamb):
    return max(np.linalg.norm(Y, ord=2), np.linalg.norm(Y, np.inf)/lamb)

In [28]:
class R_pca:

    def __init__(self, D, Y, mu=None, lmbda=None, rho=None):
        self.D = D
        self.E = np.zeros(self.D.shape)
        # self.Y = np.zeros(self.D.shape)
        self.Y = Y
        self.rho = rho

        if mu:
            self.mu = mu
        else:
            self.mu = np.prod(self.D.shape) / (4 * self.norm_p(self.D, 2))  

        self.mu_inv = 1 / self.mu

        if lmbda:
            self.lmbda = lmbda
        else:
            self.lmbda = 1 / np.sqrt(np.max(self.D.shape))

    @staticmethod
    def norm_p(M, p):
        return np.sum(np.power(M, p))

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))
        # diagE = np.diag(S)
        # np.diag(diagS(1:svp) - 1/mu)

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self, tol=None, max_iter=1000, iter_print=100):
        iter = 0
        err = np.Inf
        Ek = self.E
        Yk = self.Y
        Ak = np.zeros(self.D.shape)

        if tol:
            _tol = tol
        else:
            _tol = 1E-7 * self.norm_p(np.abs(self.D), 2)

        while (err > _tol) and iter < max_iter:
            Ak = self.svd_threshold(self.D - Ek + self.mu_inv * Yk, self.mu_inv)
            Ek = self.shrink(self.D - Ak + (self.mu_inv * Yk), self.mu_inv * self.lmbda)
            Yk = Yk + self.mu * (self.D - Ak - Ek)
            err = self.norm_p(np.abs(self.D - Ak - Ek), 2)
            self.mu *= self.rho
            iter += 1
#             if (iter % iter_print) == 0 or iter == 1 or iter > max_iter or err <= _tol:
#                 print('iteration: {0}, error: {1}'.format(iter, err))

        self.A = Ak
        self.E = Ek
        return Ak, Ek

    def plot_fit(self, size=None, tol=0.1, axis_on=True):

        n, d = self.D.shape

        if size:
            nrows, ncols = size
        else:
            sq = np.ceil(np.sqrt(n))
            nrows = int(sq)
            ncols = int(sq)

        ymin = np.nanmin(self.D)
        ymax = np.nanmax(self.D)
        print('ymin: {0}, ymax: {1}'.format(ymin, ymax))

        numplots = np.min([n, nrows * ncols])
        plt.figure()

        for n in range(numplots):
            plt.subplot(nrows, ncols, n + 1)
            plt.ylim((ymin - tol, ymax + tol))
            plt.plot(self.L[n, :] + self.S[n, :], 'r')
            plt.plot(self.L[n, :], 'b')
            if not axis_on:
                plt.axis('off')

In [42]:
def InexactALM(D, lamb, mu, rho):
    # Y0 = D/J(D, lamb)
    Y0 = np.zeros(D.shape)
    rpca = R_pca(D, Y0, mu, lamb, rho)
    A, E = rpca.fit()
    return (A, E)

In [44]:
def Clustering(D):
    n = len(D)
    lamb = 1/(32*np.sqrt(n))*2
    i = 0
    while(i < 5):
        A, E = InexactALM(D, lamb, mu=1.2, rho=1.3)
        if(A.trace() > n):
            lamb /= 2
        elif(A.trace() < n):
            lamb *= 1
        else:
            pass
        i += 1
    A = np.around(A)
    E = np.around(E)
    A = A.astype(int)
    E = E.astype(int)
    print(A)
    print(E)
    print("Rank of A:", np.linalg.matrix_rank(A))
    print("Rank of E: ", np.linalg.matrix_rank(E))

In [45]:
if __name__=="__main__":
    n = 5
    A = np.random.randint(2, size=(n, n))
    print(A)
    print(np.linalg.matrix_rank(A))
    Clustering(A)

[[0 0 0 0 1]
 [0 0 0 1 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 0 0 0 0]]
5
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[0 0 0 0 1]
 [0 0 0 1 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 0 0 0 0]]
Rank of A: 0
Rank of E:  5
